#Pré-processing des données : Indice de fragilité numérique

Avant de commencer à construire l'indice de fragilité numérique, la première étape est de mettre à jour les données utilisées et de faire les calculs nécessaires aux différentes variables de cet indice. 

In [1]:
# connexion à Google Drive pour importer les dossiers
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

## 1. Taux de plus de 60 ans, taux de ménages d'une personne et taux de familles monoparentales

In [3]:
# Chargement des données démographiques
compo_menage = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum/base-cc-coupl-fam-men-2017.CSV', sep=';')
compo_menage_2 = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum/base-cc-coupl-fam-men-2017-COM.CSV', sep=";")
compo_menage = compo_menage.append(compo_menage_2).reset_index()
compo_menage.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,CODGEO,C17_MEN,C17_MENPSEUL,C17_MENHSEUL,C17_MENFSEUL,C17_MENSFAM,C17_MENFAM,C17_MENCOUPSENF,C17_MENCOUPAENF,C17_MENFAMMONO,C17_PMEN,C17_PMEN_MENPSEUL,C17_PMEN_MENHSEUL,C17_PMEN_MENFSEUL,C17_PMEN_MENSFAM,C17_PMEN_MENFAM,C17_PMEN_MENCOUPSENF,C17_PMEN_MENCOUPAENF,C17_PMEN_MENFAMMONO,P17_POP15P,P17_POP1519,P17_POP2024,P17_POP2539,P17_POP4054,P17_POP5564,P17_POP6579,P17_POP80P,P17_POPMEN1519,P17_POPMEN2024,P17_POPMEN2539,P17_POPMEN4054,P17_POPMEN5564,P17_POPMEN6579,P17_POPMEN80P,P17_POP1519_PSEUL,P17_POP2024_PSEUL,P17_POP2539_PSEUL,P17_POP4054_PSEUL,P17_POP5564_PSEUL,...,P07_POP6579_PSEUL,P07_POP80P_PSEUL,P07_POP1519_COUPLE,P07_POP2024_COUPLE,P07_POP2539_COUPLE,P07_POP4054_COUPLE,P07_POP5564_COUPLE,P07_POP6579_COUPLE,P07_POP80P_COUPLE,P07_POP15P_MARIE,P07_POP15P_CELIB,P07_POP15P_VEUF,P07_POP15P_DIVOR,C07_MEN_CS1,C07_MEN_CS2,C07_MEN_CS3,C07_MEN_CS4,C07_MEN_CS5,C07_MEN_CS6,C07_MEN_CS7,C07_MEN_CS8,C07_PMEN_CS1,C07_PMEN_CS2,C07_PMEN_CS3,C07_PMEN_CS4,C07_PMEN_CS5,C07_PMEN_CS6,C07_PMEN_CS7,C07_PMEN_CS8,C07_FAM,C07_COUPAENF,C07_FAMMONO,C07_HMONO,C07_FMONO,C07_COUPSENF,C07_NE24F0,C07_NE24F1,C07_NE24F2,C07_NE24F3,C07_NE24F4P
0,0,1001,315.154412,71.929625,35.964813,35.964813,5.100362,238.124424,112.207972,110.689754,15.226698,774.254876,71.929625,35.964813,35.964813,10.200725,692.124526,224.415944,427.128852,40.579730,614.445217,51.313236,18.176249,106.958342,195.068521,96.818491,113.392959,32.717419,51.313236,18.176249,106.958342,195.068521,96.818491,113.392959,32.717419,0.000000,0.000000,7.192963,11.303227,13.358359,...,17.851486,8.925743,0.000000,4.958746,144.795383,137.853139,86.282181,57.521454,11.900990,364.963706,179.506605,34.711222,39.669968,7.933994,11.900990,23.801981,55.537955,15.867987,63.471949,107.108914,11.900990,27.768978,19.834984,91.240927,162.646869,51.570958,222.151821,198.349840,27.768978,238.019808,119.009904,27.768978,19.834984,7.933994,91.240927,99.174920,55.537955,63.471949,15.867987,3.966997
1,1,1002,107.160494,30.617284,25.514403,5.102881,0.000000,76.543210,35.720165,35.720165,5.102881,265.349794,30.617284,25.514403,5.102881,0.000000,234.732510,71.440329,147.983539,15.308642,191.868313,11.226337,4.082305,55.111111,51.028807,25.514403,33.679012,11.226337,11.226337,4.082305,55.111111,51.028807,25.514403,33.679012,11.226337,0.000000,0.000000,4.082305,6.123457,2.041152,...,6.882328,2.949569,0.000000,0.983190,41.293971,25.562934,16.714226,16.714226,5.899139,77.671993,51.125868,8.848708,13.764657,3.932759,3.932759,11.798277,15.731036,0.000000,15.731036,35.394832,0.000000,15.731036,15.731036,19.663796,35.394832,0.000000,39.327591,66.856905,0.000000,58.991387,31.462073,3.932759,3.932759,0.000000,23.596555,31.462073,11.798277,15.731036,0.000000,0.000000
2,2,1004,6405.910326,2659.428492,1207.832644,1451.595848,92.821886,3653.659948,1490.158562,1509.068222,654.433164,13539.100310,2659.428492,1207.832644,1451.595848,212.639705,10667.032114,3026.023009,5967.743080,1673.266025,11309.467304,1036.882405,842.752556,2983.514483,2549.754986,1551.667575,1586.176459,758.718840,950.416782,793.426970,2903.863393,2509.776506,1538.716113,1541.438539,635.564924,90.764616,154.317972,573.969561,503.884102,414.584517,...,360.782918,252.501736,28.169137,311.140466,1776.508503,1819.063085,991.680713,912.017448,211.612379,4946.599229,3640.562593,805.513617,928.018569,0.000000,176.876940,455.649696,916.598986,697.704688,1293.170060,1659.057668,266.454759,0.000000,544.329226,1364.921469,2189.104098,1538.802502,3268.330873,2755.348988,583.718310,3433.897679,1431.949014,512.331482,69.374672,442.956810,1489.617182,1632.483842,776.942502,630.333691,295.807389,98.330254
3,3,1005,636.197591,134.086315,80.451789,53.634526,5.298234,496.813042,201.268460,253.542925,42.001657,1663.687004,134.086315,80.451789,53.634526,10.596468,1519.004222,407.770727,996.206864,115.026631,1344.987086,99.149469,87.186728,290.320733,389.858664,216.354462,200.447239,61.669791,99.149469,87.186728,290.320733,389.858664,216.354462,200.447239,61.669791,0.000000,3.218072,16

In [4]:
compo_menage.shape

(35000, 238)

In [5]:
compo_menage = compo_menage[['CODGEO','C17_MEN', 'C17_MENPSEUL', 'C17_MENFAMMONO', 'P17_POP15P','P17_POP6579', 'P17_POP80P', 'P17_POP1519', 'P17_POP2024']]
compo_menage['TX_MENSEUL'] = (compo_menage['C17_MENPSEUL'] / compo_menage['C17_MEN']) * 100
compo_menage['TX_FAMMONO'] = (compo_menage['C17_MENFAMMONO'] / compo_menage['C17_MEN']) * 100
compo_menage['TX_65ETPLUS'] = ((compo_menage['P17_POP6579'] + compo_menage['P17_POP80P']) / compo_menage['P17_POP15P']) * 100
compo_menage['TX_25ETMOINS']= ((compo_menage['P17_POP1519'] + compo_menage['P17_POP2024']) / compo_menage['P17_POP15P']) * 100

In [6]:
compo_menage.head()

,CODGEO,C17_MEN,C17_MENPSEUL,C17_MENFAMMONO,P17_POP15P,P17_POP6579,P17_POP80P,P17_POP1519,P17_POP2024,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS
0,1001,315.154412,71.929625,15.226698,614.445217,113.392959,32.717419,51.313236,18.176249,22.823614,4.831504,23.779236,11.309305
1,1002,107.160494,30.617284,5.102881,191.868313,33.679012,11.226337,11.226337,4.082305,28.571429,4.761905,23.404255,7.978723
2,1004,6405.910326,2659.428492,654.433164,11309.467304,1586.176459,758.718840,1036.882405,842.752556,41.515231,10.216084,20.733915,16.620013
3,1005,636.197591,134.086315,42.001657,1344.987086,200.447239,61.669791,99.149469,87.186728,21.076206,6.601983,19.488442,13.854125
4,1006,55.500000,20.181818,5.045455,98.890909,18.163636,8.072727,3.027273,7.063636,36.363636,9.090909,26.530612,10.204082


## 2. Demandeurs d'emplois

Les fichiers utilisés ici proviennent du site de l'INSEE. Le nombre de demandeurs d'emplois est celui que l'on avait au 31 décembre 2019 (https://www.insee.fr/fr/statistiques/4633819?sommaire=2500477). Pour calculer le taux de demandeur d'emplois, on utilise le nombre de personnes comptabilisé dans la population active. Le nombre de personnes actives (entre 15 et 64 ans) provient du site de l'INSEE (chiffres de 2016).

In [7]:
# Chargement des données sur les demandeurs d'emplois et la population active
demandeur_emplois = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum//data_DEFM2019_com.csv', sep=";")
pop_active = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum//base-cc-emploi-pop-active-2016.CSV', sep=";")
pop_active2 = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum/base-cc-emploi-pop-active-2016-COM.CSV', sep=";")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Création d'un df unique avec la population active de toutes les communes (Mayotte inclus)
pop_active = pop_active.append(pop_active2).reset_index()

La population active est composée de deux fichiers : un fichier avec les chiffres de la France (sans Mayotte) et un fichier contenant les informations de Mayotte.

In [9]:
# Conservation des données de population active entre 15 et 64 ans pour l'année 2016
pop_active = pop_active[['CODGEO', 'P16_ACT1564']]
pop_active.head()

,CODGEO,P16_ACT1564
0,1001,376.000000
1,1002,123.000000
2,1004,6697.333122
3,1005,864.845592
4,1006,58.000000


On conserve donc le CODGEO (code INSEE de chaque commune) et le nombre de personnes actives âgés de 15 à 64 ans. 

Concernant le total de demandeurs d'emplois, on conserve les valeurs associées au CODGEO (code INSEE de la commune), des demandeurs d'emplois de catégorie A, B et C dans les tranches d'âge suivants : moins de 25 ans, entre 26 et 49 ans et les plus de 50 ans. Pour le calcul du taux de demandeurs d'emplois, on additionne ces valeurs que l'on divisera par le nombre de personnes dans la population active. 

In [10]:
# Conservation des données sur les demandeurs d'emplois de catégorie A, B et C. 
# Calcul du total de demandeurs d'emplois sans prise en compte de l'âge
demandeur_emplois = demandeur_emplois[['CODGEO', 'ABC_25', 'ABC_2649', 'ABC_50']]
demandeur_emplois['ABC_TOT'] = demandeur_emplois['ABC_25'] + demandeur_emplois['ABC_2649'] + demandeur_emplois['ABC_50']
demandeur_emplois = demandeur_emplois.sort_values('CODGEO')
demandeur_emplois.head()

,CODGEO,ABC_25,ABC_2649,ABC_50,ABC_TOT
0,01004,256.0,869.0,332.0,1457.0
1,01031,68.0,234.0,73.0,375.0
2,01033,210.0,903.0,357.0,1470.0
3,01034,147.0,444.0,193.0,784.0
4,01053,678.0,2441.0,1029.0,4148.0


In [11]:
pop_active.to_csv('pop_active.csv')
demandeur_emplois.to_csv('demandeur_emplois.csv')

## 3. Personnes peu ou pas diplômés

Les données utilisées proviennent du site de l'INSEE (https://www.insee.fr/fr/statistiques/4516086?sommaire=4516089). Les données proviennent de 2017 avec le découpage en vigueur au 1er janvier 2020.

In [12]:
# Chargement des données en lien avec la scolarité dans toutes les communes
scol = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum//base-cc-diplomes-formation-2017.CSV', sep=";")
scol2 = pd.read_csv('/content/drive/My Drive/Data4Good/8_Mednum//base-cc-diplomes-formation-2017-COM.CSV', sep=";")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# Création d'un df unique avec les données de Mayotte incluses
scol = scol.append(scol2).reset_index()

Le taux de personnes non scolarisés peu ou pas diplômés est calculé sur le nombre de personnes de plus de 15 ans non scolarisés avec un diplôme minimum divisé par la population totale de plus de 15 ans. 

In [14]:
# Calcul du total de personnes scolarisées
# Calcul du total de personnes de plus de 15 ans non scolarisés
# Calcul du taux de personnes de plus de 15 ans non scolarisés
scol = scol[['CODGEO', 'P17_POP1517', 'P17_POP1824', 'P17_POP2529', 'P17_POP30P', 'P17_NSCOL15P', 'P17_NSCOL15P_DIPLMIN','P17_NSCOL15P_BEPC','P17_NSCOL15P_CAPBEP' ,'P17_NSCOL15P_BAC']]
scol['TOT_POP15P'] = scol['P17_POP1517'] + scol['P17_POP1824'] + scol['P17_POP2529'] + scol['P17_POP30P']
scol['TOT_NSCOL15P'] = scol['P17_NSCOL15P_DIPLMIN'] + scol['P17_NSCOL15P_BEPC'] + scol['P17_NSCOL15P_CAPBEP']
scol['TX_NSCOL15P'] = (scol['TOT_NSCOL15P']/scol['TOT_POP15P']) * 100
scol.head()

,CODGEO,P17_POP1517,P17_POP1824,P17_POP2529,P17_POP30P,P17_NSCOL15P,P17_NSCOL15P_DIPLMIN,P17_NSCOL15P_BEPC,P17_NSCOL15P_CAPBEP,P17_NSCOL15P_BAC,TOT_POP15P,TOT_NSCOL15P,TX_NSCOL15P
0,1001,33.181349,36.308136,33.410552,511.545180,558.105307,107.168650,26.381412,186.909421,95.095609,614.445217,320.459483,52.154281
1,1002,7.144033,8.164609,13.267490,163.292181,178.600823,22.452675,7.144033,47.967078,38.781893,191.868313,77.563786,40.425532
2,1004,593.927796,1285.707165,1108.451063,8321.381280,10149.026196,2207.010696,588.550154,2799.751384,1938.190106,11309.467304,5595.312233,49.474587
3,1005,59.940064,126.396133,73.152717,1085.498171,1234.359736,294.111609,55.410401,371.050484,221.492025,1344.987086,720.572494,53.574678
4,1006,3.027273,7.063636,6.054545,82.745455,95.863636,17.154545,4.036364,31.281818,30.272727,98.890909,52.472727,53.061224


In [15]:
scol.to_csv('scol.csv')

## 4. Assemblage des fichiers

On effectue une jointure entre le fichier contenant les informations démographiques et celles des demandeurs d'emplois en utilisant le "CODGEO" de chacune de nos tables. 

In [16]:
df1 = pd.merge(compo_menage, demandeur_emplois, on = ['CODGEO'], how='outer')
df1.head()

,CODGEO,C17_MEN,C17_MENPSEUL,C17_MENFAMMONO,P17_POP15P,P17_POP6579,P17_POP80P,P17_POP1519,P17_POP2024,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,ABC_25,ABC_2649,ABC_50,ABC_TOT
0,1001,315.154412,71.929625,15.226698,614.445217,113.392959,32.717419,51.313236,18.176249,22.823614,4.831504,23.779236,11.309305,NaN,NaN,NaN,NaN
1,1002,107.160494,30.617284,5.102881,191.868313,33.679012,11.226337,11.226337,4.082305,28.571429,4.761905,23.404255,7.978723,NaN,NaN,NaN,NaN
2,1004,6405.910326,2659.428492,654.433164,11309.467304,1586.176459,758.718840,1036.882405,842.752556,41.515231,10.216084,20.733915,16.620013,NaN,NaN,NaN,NaN
3,1005,636.197591,134.086315,42.001657,1344.987086,200.447239,61.669791,99.149469,87.186728,21.076206,6.601983,19.488442,13.854125,NaN,NaN,NaN,NaN
4,1006,55.500000,20.181818,5.045455,98.890909,18.163636,8.072727,3.027273,7.063636,36.363636,9.090909,26.530612,10.204082,NaN,NaN,NaN,NaN


On supprime les colonnes qui ne seront pas nécessaires à la suite du traitement.

In [17]:
df1 = df1.drop(columns=['C17_MEN', 'C17_MENPSEUL', 'C17_MENFAMMONO', 'P17_POP15P','P17_POP6579', 'P17_POP80P','ABC_25', 'ABC_2649', 'ABC_50', 'P17_POP1519', 'P17_POP2024'])
df1.head()

,CODGEO,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,ABC_TOT
0,1001,22.823614,4.831504,23.779236,11.309305,NaN
1,1002,28.571429,4.761905,23.404255,7.978723,NaN
2,1004,41.515231,10.216084,20.733915,16.620013,NaN
3,1005,21.076206,6.601983,19.488442,13.854125,NaN
4,1006,36.363636,9.090909,26.530612,10.204082,NaN


In [18]:
df1.shape

(36109, 6)

In [19]:
del compo_menage
del demandeur_emplois

On effectue une jointure entre le fichier contenant les informations sur la population non scolarisés et celles de la population active en utilisant le "CODGEO" de chacune de nos tables. 

In [20]:
df2 = pd.merge(scol, pop_active, on = 'CODGEO', how='outer')
df2.head()

,CODGEO,P17_POP1517,P17_POP1824,P17_POP2529,P17_POP30P,P17_NSCOL15P,P17_NSCOL15P_DIPLMIN,P17_NSCOL15P_BEPC,P17_NSCOL15P_CAPBEP,P17_NSCOL15P_BAC,TOT_POP15P,TOT_NSCOL15P,TX_NSCOL15P,P16_ACT1564
0,1001,33.181349,36.308136,33.410552,511.545180,558.105307,107.168650,26.381412,186.909421,95.095609,614.445217,320.459483,52.154281,376.000000
1,1002,7.144033,8.164609,13.267490,163.292181,178.600823,22.452675,7.144033,47.967078,38.781893,191.868313,77.563786,40.425532,123.000000
2,1004,593.927796,1285.707165,1108.451063,8321.381280,10149.026196,2207.010696,588.550154,2799.751384,1938.190106,11309.467304,5595.312233,49.474587,6697.333122
3,1005,59.940064,126.396133,73.152717,1085.498171,1234.359736,294.111609,55.410401,371.050484,221.492025,1344.987086,720.572494,53.574678,864.845592
4,1006,3.027273,7.063636,6.054545,82.745455,95.863636,17.154545,4.036364,31.281818,30.272727,98.890909,52.472727,53.061224,58.000000


On supprime les différentes colonnes non utilisées.

In [21]:
df2 = df2.drop(columns=['P17_POP1517', 'P17_POP1824', 'P17_POP2529','P17_POP30P', 'P17_NSCOL15P_BEPC', 'P17_NSCOL15P_CAPBEP', 'P17_NSCOL15P_BAC'])
df2.head()

,CODGEO,P17_NSCOL15P,P17_NSCOL15P_DIPLMIN,TOT_POP15P,TOT_NSCOL15P,TX_NSCOL15P,P16_ACT1564
0,1001,558.105307,107.168650,614.445217,320.459483,52.154281,376.000000
1,1002,178.600823,22.452675,191.868313,77.563786,40.425532,123.000000
2,1004,10149.026196,2207.010696,11309.467304,5595.312233,49.474587,6697.333122
3,1005,1234.359736,294.111609,1344.987086,720.572494,53.574678,864.845592
4,1006,95.863636,17.154545,98.890909,52.472727,53.061224,58.000000


In [22]:
df2 = df2.merge(df1, on='CODGEO', how='outer')
df2.head()

,CODGEO,P17_NSCOL15P,P17_NSCOL15P_DIPLMIN,TOT_POP15P,TOT_NSCOL15P,TX_NSCOL15P,P16_ACT1564,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,ABC_TOT
0,1001,558.105307,107.168650,614.445217,320.459483,52.154281,376.000000,22.823614,4.831504,23.779236,11.309305,NaN
1,1002,178.600823,22.452675,191.868313,77.563786,40.425532,123.000000,28.571429,4.761905,23.404255,7.978723,NaN
2,1004,10149.026196,2207.010696,11309.467304,5595.312233,49.474587,6697.333122,41.515231,10.216084,20.733915,16.620013,NaN
3,1005,1234.359736,294.111609,1344.987086,720.572494,53.574678,864.845592,21.076206,6.601983,19.488442,13.854125,NaN
4,1006,95.863636,17.154545,98.890909,52.472727,53.061224,58.000000,36.363636,9.090909,26.530612,10.204082,NaN


In [23]:
df2 = df2.drop(columns = ['P17_NSCOL15P','P17_NSCOL15P_DIPLMIN', 'TOT_POP15P', 'TOT_NSCOL15P'])
df2.head()

,CODGEO,TX_NSCOL15P,P16_ACT1564,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,ABC_TOT
0,1001,52.154281,376.000000,22.823614,4.831504,23.779236,11.309305,NaN
1,1002,40.425532,123.000000,28.571429,4.761905,23.404255,7.978723,NaN
2,1004,49.474587,6697.333122,41.515231,10.216084,20.733915,16.620013,NaN
3,1005,53.574678,864.845592,21.076206,6.601983,19.488442,13.854125,NaN
4,1006,53.061224,58.000000,36.363636,9.090909,26.530612,10.204082,NaN


In [24]:
del scol
del pop_active

## Dataset final

In [25]:
data = df2
data['TX_DEMANDEUR_EMPLOIS'] = (data['ABC_TOT']/data['P16_ACT1564'])*100
data = data.drop(columns=['ABC_TOT', 'P16_ACT1564'])
data.head()

,CODGEO,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,TX_DEMANDEUR_EMPLOIS
0,1001,52.154281,22.823614,4.831504,23.779236,11.309305,NaN
1,1002,40.425532,28.571429,4.761905,23.404255,7.978723,NaN
2,1004,49.474587,41.515231,10.216084,20.733915,16.620013,NaN
3,1005,53.574678,21.076206,6.601983,19.488442,13.854125,NaN
4,1006,53.061224,36.363636,9.090909,26.530612,10.204082,NaN


In [26]:
# boucle pour ajouter un 0 au début des codes insee de 4 chiffres
codgeo = []
for value in data["CODGEO"]:
    value = str(value)
    codgeo.append(value.zfill(5))
len(codgeo)
data["CODE_INSEE"] = codgeo

In [27]:
data.drop("CODGEO", axis = 1, inplace = True)

In [28]:
data = data[['CODE_INSEE', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS','TX_DEMANDEUR_EMPLOIS']]
data.head()

,CODE_INSEE,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,TX_DEMANDEUR_EMPLOIS
0,01001,52.154281,22.823614,4.831504,23.779236,11.309305,NaN
1,01002,40.425532,28.571429,4.761905,23.404255,7.978723,NaN
2,01004,49.474587,41.515231,10.216084,20.733915,16.620013,NaN
3,01005,53.574678,21.076206,6.601983,19.488442,13.854125,NaN
4,01006,53.061224,36.363636,9.090909,26.530612,10.204082,NaN


In [29]:
dir = '/content/drive/My Drive/Data4Good/8_Mednum/'
data.to_csv(f'{dir}data_scol_menage_demandeurEmplois.csv')

In [30]:
data.shape

(36114, 7)